In [ ]:
#https://towardsdatascience.com/ai-generates-taylor-swifts-song-lyrics-6fd92a03ef7e


In [1]:
import numpy as np
import pandas as pd
import sys 
from keras.models import Sequential
from keras.layers import LSTM, Activation, Flatten, Dropout, Dense, Embedding, TimeDistributed
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
#Load the dataset
dataset = pd.read_csv("taylor-swift-song-lyrics-from-all-the-albums/taylor_swift_lyrics.csv", encoding = "latin1")
dataset.head()

,artist,album,track_title,track_n,lyric,line,year
0,Taylor Swift,Taylor Swift,Tim McGraw,1,He said the way my blue eyes shined,1,2006
1,Taylor Swift,Taylor Swift,Tim McGraw,1,Put those Georgia stars to shame that night,2,2006
2,Taylor Swift,Taylor Swift,Tim McGraw,1,"I said, ""That's a lie""",3,2006
3,Taylor Swift,Taylor Swift,Tim McGraw,1,Just a boy in a Chevy truck,4,2006
4,Taylor Swift,Taylor Swift,Tim McGraw,1,That had a tendency of gettin' stuck,5,2006


In [4]:
def processFirstLine(lyrics, songID, songName, row):
    lyrics.append(row['lyric'] + '\n')
    songID.append( row['year']*100+ row['track_n'])
    songName.append(row['track_title'])
    return lyrics,songID,songName
# define empty lists for the lyrics , songID , songName 
lyrics = []
songID = []
songName = []
# songNumber indicates the song number in the dataset
songNumber = 1
# i indicates the song number
i = 0
isFirstLine = True
# Iterate through every lyrics line and join them together for each song independently 
for index,row in dataset.iterrows():
    if(songNumber == row['track_n']):
        if (isFirstLine):
            lyrics,songID,songName = processFirstLine(lyrics,songID,songName,row)
            isFirstLine = False
        else :
            #if we still in the same song , keep joining the lyrics lines    
            lyrics[i] +=  row['lyric'] + '\n'
    #When it's done joining a song's lyrics lines , go to the next song :    
    else :
        lyrics,songID,songName = processFirstLine(lyrics,songID,songName,row)
        songNumber = row['track_n']
        i+=1

In [5]:
lyrics_data = pd.DataFrame({'songID':songID, 'songName':songName, 'lyrics':lyrics })

In [6]:
lyrics_data.head(20)

,songID,songName,lyrics
0,200601,Tim McGraw,He said the way my blue eyes shined\nPut those...
1,200602,Picture To Burn,"State the obvious, I didn't get my perfect fan..."
2,200603,Teardrops On My Guitar,Drew looks at me\nI fake a smile so he won't s...
3,200604,A Place In This World,"I don't know what I want, so don't ask me\nCau..."
4,200605,Cold as You,You have a way of coming easily to me\nAnd whe...
5,200606,The Outside,I didn't know what I would find\nWhen I went l...
6,200607,Tied Together With A Smile,Seems the only one who doesn't see your beauty...
7,200608,Stay Beautiful,"Cory's eyes are like a jungle\nHe smiles, it's..."
8,200609,Should've Said No,It's strange to think the songs we used to sin...
9,200610,Mary's Song (Oh My My My),"She said, I was seven and you were nine\nI loo..."


In [8]:
# Save Lyrics in .txt file
with open('document.pdf', 'w',encoding="utf-8") as filehandle:  
    for listitem in lyrics:
        filehandle.write('%s\n' % listitem)

In [9]:
# Load the dataset and convert it to lowercase :
textFileName = 'lyricsText.txt'
raw_text = open(textFileName, encoding = 'UTF-8').read()
raw_text = raw_text.lower()

In [10]:
raw_text

'he said the way my blue eyes shined\nput those georgia stars to shame that night\ni said, "that\'s a lie"\njust a boy in a chevy truck\nthat had a tendency of gettin\' stuck\non backroads at night\nand i was right there beside him all summer long\nand then the time we woke up to find that summer gone\nbut when you think tim mcgraw\ni hope you think my favorite song\nthe one we danced to all night long\nthe moon like a spotlight on the lake\nwhen you think happiness\ni hope you think that little black dress\nthink of my head on your chest\nand my old faded blue jeans\nwhen you think tim mcgraw\ni hope you think of me\nseptember saw a month of tears\nand thankin\' god that you weren\'t here\nto see me like that\nbut in a box beneath my bed\nis a letter that you never read\nfrom three summers back\nit\'s hard not to find it all a little bittersweet\nand lookin\' back on all of that, it\'s nice to believe\nwhen you think tim mcgraw\ni hope you think my favorite song\nthe one we danced to 

In [11]:
# Mapping chars to ints :
chars = sorted(list(set(raw_text)))
int_chars = dict((i, c) for i, c in enumerate(chars))
chars_int = dict((i, c) for c, i in enumerate(chars))

In [12]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters : " , n_chars) # number of all the characters in lyricsText.txt
print("Total Vocab : ", n_vocab) # number of unique characters


Total Characters :  173698
Total Vocab :  58


In [13]:
# process the dataset:
seq_len = 100
data_X = []
data_y = []
for i in range(0, n_chars - seq_len, 1):
    # Input Sequeance(will be used as samples)
    seq_in  = raw_text[i:i+seq_len]
    # Output sequence (will be used as target)
    seq_out = raw_text[i + seq_len]
    # Store samples in data_X
    data_X.append([chars_int[char] for char in seq_in])
    # Store targets in data_y
    data_y.append(chars_int[seq_out])
n_patterns = len(data_X)
print( 'Total Patterns : ', n_patterns)

Total Patterns :  173598


In [14]:
# Reshape X to be suitable to go into LSTM RNN :
X = np.reshape(data_X , (n_patterns, seq_len, 1))
# Normalizing input data :
X = X/ float(n_vocab)
# One hot encode the output targets :
y = np_utils.to_categorical(data_y)


In [15]:
LSTM_layer_num = 4 # number of LSTM layers
layer_size = [256,256,256,256] # number of nodes in each layer


In [16]:
model = Sequential()

W1010 12:19:48.944770 69104 deprecation_wrapper.py:119] From C:\Users\20131073\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [17]:
model.add(LSTM(layer_size[0], input_shape =(X.shape[1], X.shape[2]), return_sequences = True))

W1010 12:19:51.261592 69104 deprecation_wrapper.py:119] From C:\Users\20131073\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1010 12:19:51.264585 69104 deprecation_wrapper.py:119] From C:\Users\20131073\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [18]:
for i in range(1,LSTM_layer_num) :
    model.add(LSTM(layer_size[i], return_sequences=True))

In [19]:
model.add(Flatten())

In [20]:
model.add(Dense(y.shape[1]))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

W1010 12:20:15.497399 69104 deprecation_wrapper.py:119] From C:\Users\20131073\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1010 12:20:15.519342 69104 deprecation_wrapper.py:119] From C:\Users\20131073\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3576: The name tf.log is deprecated. Please use tf.math.log instead.



In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 256)          525312    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100, 256)          525312    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100, 256)          525312    
_________________________________________________________________
flatten_1 (Flatten)          (None, 25600)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 58)                1484858   
_________________________________________________________________
activation_1 (Activation)    (None, 58)               

In [22]:
# Configure the checkpoint :
checkpoint_name = 'saved_models/Weights-LSTM-improvement-{epoch:03d}-{loss:.5f}-bigger.hdf5'
checkpoint = ModelCheckpoint(checkpoint_name, monitor='loss', verbose = 1, save_best_only = True, mode ='min')
callbacks_list = [checkpoint]

In [24]:
# Fit the model :
model_params = {'epochs':8,
                'batch_size':128,
                'callbacks':callbacks_list,
                'verbose':1,
                'validation_split':0.2,
                'validation_data':None,
                'shuffle': True,
                'initial_epoch':0,
                'steps_per_epoch':None,
                'validation_steps':None}
model.fit(X,
          y,
          epochs = model_params['epochs'],
           batch_size = model_params['batch_size'],
           callbacks= model_params['callbacks'],
           verbose = model_params['verbose'],
           validation_split = model_params['validation_split'],
           validation_data = model_params['validation_data'],
           shuffle = model_params['shuffle'],
           initial_epoch = model_params['initial_epoch'],
           steps_per_epoch = model_params['steps_per_epoch'],
           validation_steps = model_params['validation_steps'])

Train on 138878 samples, validate on 34720 samples
Epoch 1/8
138878/138878 [==============================] - 3491s 25ms/step - loss: 3.0132 - val_loss: 3.0251

Epoch 00001: loss improved from inf to 3.01318, saving model to saved_models/Weights-LSTM-improvement-001-3.01318-bigger.hdf5
Epoch 2/8
138878/138878 [==============================] - 3648s 26ms/step - loss: 3.0098 - val_loss: 3.0237

Epoch 00002: loss improved from 3.01318 to 3.00981, saving model to saved_models/Weights-LSTM-improvement-002-3.00981-bigger.hdf5
Epoch 3/8
109056/138878 [======================>.......] - ETA: 30:51 - loss: 3.0082

KeyboardInterrupt: 

In [25]:
wights_file = 'saved_models/Weights-LSTM-improvement-002-3.00981-bigger.hdf5' # weights file path
model.load_weights(wights_file)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [27]:
# set a random seed :
start = np.random.randint(0, len(data_X)-1)
pattern = data_X[start]
print('Seed : ')
print("\"",''.join([int_chars[value] for value in pattern]), "\"\n")
# How many characters you want to generate
generated_characters = 1000
# Generate Charachters :
for i in range(generated_characters):
    x = np.reshape(pattern, ( 1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x,verbose = 0)
    index = np.argmax(prediction)
    result = int_chars[index]
    #seq_in = [int_chars[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print('\nDone')

Seed : 
"  you, i loved you from the very first day
i watch superman fly away
you've got a busy day today
go s "

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      